# TP1 INFO8003
The idea behind this notebook is to get familiar with basic concepts of reinforcement learning such as the design of environments and the state value function $V^\pi(s)$. 


## Question 1: Implementing a Discrete Gold Miner Environment

Your task is to create a discrete environment for a "Gold Miner" game following the [Gym API](https://gymnasium.farama.org/v0.26.3/api/env/). Specifically, you need to implement the `step` and `reset` functions:

- **`step(action)`**:  
  This method updates the environment based on the given action. It should return:  
  1. The agent's next observation.  
  2. The reward obtained for the action.  
  3. Whether the episode has ended (`terminated`) or paused (`truncated`).  
  4. Additional information, such as metrics or debugging data about the environment.  

- **`reset()`**:  
  This method initializes the environment to its starting state. It should return:  
  1. The agent's first observation for a new episode.  
  2. Additional information, such as metrics or debugging data.  

---

### Gold Miner Environment

You get to visit a gold mine and you are allowed to spend as much time in it as you wish to get as much gold as possible. Working and spending time in the mine is complicated and rather than maximizing the quantity of gold mined, you want to optimize your satisfaction. You don't plan to stay in the mine for very long. Since you have followed the RL course last year, you have computed a discount factor of $95\%$ which allows you to not spend too much time in the gold mine.

You start at the entry of the mine, at the ground level. You can decide to either go one level down, stay where you are or go one level up. To get to the gold, you have to dig trough several layers of soil. Staying at ground level, does not give you any satisfaction. The first two layers cost you one point of satisfaction each. The third layer contains a little gold and rewards you with one satisfaction point. If you decide to dig deeper, the next 4 layers are more complicated. Each layer further would respectively cost you 1,2,3 and 4 satisfaction points. However, the last layer is full of gold and digging there rewards you with 10 satisfaction points. If you decide to stay on a layer, you keep digging the same layer and it provides the same satisfaction over and over again.

![Gold Miner Environment](Mine.png)


In [1]:
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Discrete, Tuple

class MinerGymEnv(Env):
    def __init__(self):
        """
        Initialize the Gold Miner environment.
        """
        self.action_space = Discrete(n=3, start=-1)
        self.observation_space = Discrete(9)
        self.current_state = 0
        self.discount_factor = 0.95
        self.reward = [0, -1, -1, 1, -1, -2, -3, -4, 10]    # Reward for each state by index

    def step(self, action):
        """
        Perform an action in the environment and update its state.

        Parameters:
        action (int): The action taken by the agent. This could be moving up, down, or staying at the current level.

        Returns:
        tuple: A tuple containing:
            - next_state (int): The new state of the environment after the action.
            - reward (float): The reward received for taking the action.
            - terminated (bool): Whether the episode has ended.
            - truncated (bool): Whether the episode was truncated (paused).
            - info (dict): Additional information, such as metrics or debugging data.
        """
        next_state = min(8, max(0, self.current_state + action))
        if next_state not in self.observation_space:
            raise ValueError("Invalid action: the action should lead to a valid state in the environment.")

        reward = self.reward[next_state]
        terminated = next_state == 8
        truncated = False
        self.current_state = next_state

        return next_state, reward, terminated, truncated, {}

    def reset(self):
        """
        Reset the environment to its initial state.

        This method is used to start a new episode by resetting the environment's state
        to the initial state (ground level).

        Returns:
        tuple: A tuple containing:
            - current_state (int): The initial state of the environment.
            - info (dict): Additional information, such as metrics or debugging data.
        """
        self.current_state = 0

        return self.current_state, {}

    def value_function(self, agent, n):
        """
        Estimate the state value function for a given policy over a specified number of iterations.

        Parameters:
        agent (Agent): The agent whose policy is used to estimate the value function.
        n (int): The number of iterations to perform for estimating the value function.

        Returns:
        np.ndarray: An array representing the estimated value of each state, indicating the expected cumulative reward for following the policy from each state.
        """
        state_value_function_table = np.zeros(9)
        for _ in range(n):
            for state in range(9):
                if state == 8:
                    state_value_function_table[state] = self.reward[state] # Reward = 10
                else:
                    policy = agent.chose_action(state)
                    next_state = state + policy
                    if 0 <= next_state < len(state_value_function_table):
                        state_value_function_table[state] = self.reward[state] + self.discount_factor * state_value_function_table[next_state]
                    else:
                        state_value_function_table[state] = self.reward[state] # Handle out-of-bounds by only considering the immediate reward

        return state_value_function_table



## Question 2
You are asked to code an agent that choses the always-dig policy via the function: 
- chose_action(state): that takes a state as input and returns an always-dig action

In [2]:
class Agent:
    def __init__(self):
        """
        Initialize the agent.
        """
        self.action = 1

    def chose_action(self,state):
        """
        Determine the action to take based on the current state.

        Parameters:
        state (int): The current state of the environment.

        Returns:
        int: The action chosen by the agent. In this implementation, the agent always returns
        a predefined policy of always digging.
        """
        return self.action


### Test your implementation

Interaction between agent and domain over 10 timesteps

In [3]:

env = MinerGymEnv()
state, info = env.reset()
agent = Agent()
steps = 10
trajectory = []

for _ in range(steps):
    action = agent.chose_action(state)
    next_state, reward, _, _, _ = env.step(action)
    trajectory.append((state, action, reward, next_state))
    state = next_state

for i, sample in enumerate(trajectory):
    print(sample)

(0, 1, -1, 1)
(1, 1, -1, 2)
(2, 1, 1, 3)
(3, 1, -1, 4)
(4, 1, -2, 5)
(5, 1, -3, 6)
(6, 1, -4, 7)
(7, 1, 10, 8)
(8, 1, 10, 8)
(8, 1, 10, 8)


## Question 3
Implement the state value function that evalues the estimated reward if a policy is applied for all states.

In [4]:
env.value_function(agent, 100000)

array([-1.72161367, -1.81222491, -0.85497359,  0.15265937, -0.8919375 ,
        0.11375   ,  2.225     ,  5.5       , 10.        ])

## Question 4
You will find here below the implementation of the grid world environment.

In [5]:
class GridWorld(Env):
    def __init__(self, grid_rewards, state_0= (3, 0), deterministic=True):
        """
        Initialize the GridWorld environment.

        Parameters:
        grid_rewards (list of lists): A 2D list representing the reward values for each cell in the grid.
        state_0 (tuple): The initial state of the agent in the grid, default is (3, 0).
        deterministic (bool): If True, the environment behaves deterministically; otherwise, it behaves stochastically.
        """
        self.grid = np.array(grid_rewards)
        self.n, self.m = self.grid.shape
        self.action_space = Discrete(4)  # Actions: 0=Right, 1=Left, 2=Up, 3=Down
        self.observation_space = Tuple((Discrete(self.n), Discrete(self.m)))
        self.state = state_0
        self.deterministic = deterministic
        self.gamma=0.99

    def step(self, action):
        """
        Execute an action in the environment and update the agent's state.

        Parameters:
        action (int): The action to be taken by the agent. Actions are encoded as integers:
                    0=Right, 1=Left, 2=Up, 3=Down.

        Returns:
        tuple: A tuple containing:
            - next_state (tuple): The new state of the agent after the action.
            - reward (float): The reward received for taking the action.
            - terminated (bool): Always False, as the environment does not have terminal states.
            - truncated (bool): Always False, as the environment does not have truncated states.
            - info (dict): An empty dictionary for additional information.
        """
        actions = {
            0: (0, 1),   # Right
            1: (0, -1),  # Left
            2: (-1, 0),  # Up
            3: (1, 0)    # Down
        }

        i, j = actions[action]
        x, y = self.state

        if self.deterministic:
            next_state = (
                min(max(x + i, 0), self.n - 1),
                min(max(y + j, 0), self.m - 1)
            )
        else:
            if np.random.uniform() <= 0.5:
                next_state = (
                    min(max(x + i, 0), self.n - 1),
                    min(max(y + j, 0), self.m - 1)
                )
            else:
                next_state = (0, 0)

        reward = self.grid[next_state]
        self.state = next_state

        return next_state, reward, False, False, {}

    def reset(self):
        """
        Reset the environment to the initial state.

        Returns:
        tuple: The initial state of the environment.
        """
        self.state = (3, 0)
        return self.state

    def render(self):
        """
        Display the current state of the grid with the agent's position marked.

        The grid is displayed with '.' for empty cells and 'A' for the agent's current position.
        """
        grid_display = np.zeros_like(self.grid, dtype=str)
        grid_display[:] = '.'
        x, y = self.state
        grid_display[x, y] = 'A'
        print('\n'.join(' '.join(row) for row in grid_display))

grid_rewards = [
    [-3, 1, -5, 0, 19],
    [6, 3, 8, 9, 10],
    [5, -8, 4, 1, -8],
    [6, -9, 4, 19, -5],
    [-20, -17, -4, -3, 9]
]



Implement the rule-based policy "always go right". Simulate the policy in the domain through a single trajectory of 10 steps, starting by the initial state s0 = (3, 0). Display the trajectories as a sequence of four tuples (s0, a0, r0, s1), ... , (s10, a10, r10, s11).

In [6]:
def rule_based_policy(state):
    """
    Define a rule-based policy for the agent.

    Parameters:
    state (tuple): The current state of the environment.

    Returns:
    int: The action chosen by the policy.
    """
    return 0

def simulate_policy(env, policy, steps=10):
    """
    Simulate a policy in the given environment over a specified number of steps.

    Parameters:
    env (Env): The environment in which the policy is to be simulated.
    policy (function): A function that takes a state as input and returns an action.
    steps (int): The number of steps to simulate the policy for.

    Returns:
    list: A list of tuples representing the trajectory, where each tuple contains:
          - state (tuple): The state before taking the action.
          - action (int): The action taken.
          - reward (float): The reward received after taking the action.
          - next_state (tuple): The state after taking the action.
    """
    s = env.reset()
    trajectory = []

    for _ in range(steps):
        a = policy(s)
        s_next, r, _, _, _ = env.step(a)
        trajectory.append((s, a, r, s_next))
        s = s_next

    return trajectory

env = GridWorld(grid_rewards, deterministic=True)
# Simulate trajectory
trajectory = simulate_policy(env, rule_based_policy, steps=10)
for t in trajectory:
    s, a, r, s_next = t
    print(f"State: {s}, Action: {a}, Reward: {r}, Next State: {s_next}")

State: (3, 0), Action: 0, Reward: -9, Next State: (3, 1)
State: (3, 1), Action: 0, Reward: 4, Next State: (3, 2)
State: (3, 2), Action: 0, Reward: 19, Next State: (3, 3)
State: (3, 3), Action: 0, Reward: -5, Next State: (3, 4)
State: (3, 4), Action: 0, Reward: -5, Next State: (3, 4)
State: (3, 4), Action: 0, Reward: -5, Next State: (3, 4)
State: (3, 4), Action: 0, Reward: -5, Next State: (3, 4)
State: (3, 4), Action: 0, Reward: -5, Next State: (3, 4)
State: (3, 4), Action: 0, Reward: -5, Next State: (3, 4)
State: (3, 4), Action: 0, Reward: -5, Next State: (3, 4)


## Question 5

Implement a routine to estimate $V^μ_N$ in the deterministic domain, where $\mu : S \rightarrow A$ is a stationary policy. Test your implementation with your rule-based policy of Question 4. 

Provide a bound on the suboptimality of $\mu^*_{100}$ with respect to $µ^*$. Is the bound a good
one? Compute the value of N such that this bound is equal to 0.01. Motivate your choice using the discount factor $\gamma$.

Display $V^μ_N(s)$ for each state $s$.

Reminder: $V^μ_N(s) = r(s,a) + \gamma * V^μ_{N-1}(f(s, a))\quad \forall N \ge 1$ with $V^μ_0(s) = 0$ in a deterministic domain

In [7]:
def truncated_value_function(env, policy, gamma=0.99, N=100):
    """
    Estimate the state value function using a truncated approach over a specified number of iterations.

    This function calculates the expected value of each state in the environment when following a given policy,
    using a deterministic approach to account for the environment's dynamics.

    Parameters:
    env (Env): The environment in which the policy is evaluated.
    policy (function): A function that takes a state as input and returns an action according to the policy being evaluated.
    gamma (float, optional): The discount factor, which determines the present value of future rewards. Default is 0.99.
    N (int, optional): The number of iterations to perform for estimating the value function. Default is 100.

    Returns:
    np.ndarray: An array representing the estimated value of each state, indicating the expected cumulative reward for following the policy from each state.
    """
    state_value_function_table = np.zeros((env.n, env.m))
    for _ in range(N):
        # Update value for each state
        for i in range(env.n):
            for j in range(env.m):
                s = (i, j)
                a = policy(s)
                s_next, r, _, _, _ = env.step(a)
                x, y = s_next
                state_value_function_table[i, j] = r + gamma * state_value_function_table[x, y]

    return state_value_function_table

env.reset()
truncated_value_function_table = truncated_value_function(env, rule_based_policy)
print("Truncated V with N=100:")
print(truncated_value_function_table)

Truncated V with N=100:
[[-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-318.81399107 -318.81399107 -318.81399107 -318.81399107 -318.81399107]]


> **Answer:** The suboptimality of the truncated value function $V^{\mu^*_{100}}$ with respect to the true value function $V^{\mu^*}$ can be bounded by the following inequality:
>
> $||V^{\mu^*} - V^{\mu^*{100}}||_\infty \leq \frac{2 \gamma^N B_r}{(1 - \gamma)^2}$
>
> This bound indicates that the difference between the true value function and the truncated value function decreases exponentially with the number of iterations (N).

In [27]:
gamma = 0.95
B_r = 20 # suppose reward is bounded by 20

# => error_bound = 0.01
# <=> N = \frac{\ln(\frac{0.01(1-\gamma)^2}{2B_r})}{\ln(\gamma)}

N = np.log((0.01 * (1 - gamma)**2) / (2 * B_r)) / np.log(gamma)
N = np.ceil(N)
print("Bound:", N)

# Else:
# N = 0
# error_bound = 2 * gamma**N * B_r / ((1 - gamma)**2)

# # Find N such that the error bound is equal to 0.01
# while error_bound > 0.01:
#     N += 1
#     error_bound = 2 * gamma**N * B_r / ((1 - gamma)**2)

# print("Bound:", N)

Bound: 279.0


> **Answer:** The bound here is good because it shows that the error decreases exponentially with $N$. However, it will depend on the value of $\gamma$. In fact, the discount factor $\gamma$ plays a crucial role in determining the convergence rate of the value function. 
>
> - A higher $\gamma$ means future rewards are valued more, leading to slower convergence and requiring more iterations to achieve a small error. 
> - A lower $\gamma$ values immediate rewards more, leading to faster convergence.
>
> Choosing $\gamma = 0.95$ provides a reasonable bound ($N=279$). On the contrary, setting $\gamma = 0.99999$ leads to $N = 3131975$

# Question 6
Estimate the expectation of the state value function with 10 runs using Monte-carlo estimation.

Reminder: $V^μ_N(s) = \mathbb{E}_w\{r(s,a,w) + \gamma * V^μ_{N-1}(f(s,a,w))\} \quad \forall N \ge 1$ with $V^μ_0(s) = 0$

In [8]:
def truncated_value_function_stochastic(env, policy, gamma=0.99, N=100):
    """
    Estimate the state value function using a stochastic approach over a specified number of iterations.
    This function calculates the expected value of each state in the environment when following a given policy,
    using a Monte Carlo method to account for stochasticity in the environment.

    Parameters:
        env (Env): The environment in which the policy is evaluated.
        policy (function): A function that takes a state as input and returns an action according to the policy being evaluated.
        gamma (float, optional): The discount factor, which determines the present value of future rewards. Default is 0.99.
        N (int, optional): The number of iterations to perform for estimating the value function. Default is 100.

    Returns:
        tuple: A tuple containing:
            - state_value_function_table (np.ndarray): An array representing the estimated value of each state.
            - std_state_value_function_table (np.ndarray): An array representing the standard deviation of the estimated value for each state.
    """
    state_value_function_table = np.zeros((env.n, env.m))
    std_state_value_function_table = np.zeros((env.n, env.m))

    num_runs = 10  # Monte Carlo runs per state

    # Perform N iterations of value function updates
    for _ in range(N):
        new_value_table = np.zeros_like(state_value_function_table)
        new_std_table = np.zeros_like(std_state_value_function_table)

        # Loop over all states in the grid
        for i in range(env.n):
            for j in range(env.m):
                s = (i, j)
                returns = []

                # Run Monte Carlo sampling (10 runs) for each state s
                for _ in range(num_runs):
                    a = policy(s)
                    s_next, r, _, _, _ = env.step(a)
                    x, y = s_next
                    value_next = state_value_function_table[x, y]
                    returns.append(r + gamma * value_next)

                # Estimate the expected value and its standard deviation from the 10 samples
                new_value_table[i, j] = np.mean(returns)
                new_std_table[i, j] = np.std(returns)

        # Update the value function table for the next iteration
        state_value_function_table = new_value_table
        std_state_value_function_table = new_std_table

    return state_value_function_table, std_state_value_function_table

env = GridWorld(grid_rewards, deterministic=True)
truncated_value_function_table, std_truncated_value_function_table = truncated_value_function_stochastic(env, rule_based_policy)
print("Truncated V with N=100:")
print(truncated_value_function_table)
print(std_truncated_value_function_table)

Truncated V with N=100:
[[-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]]
[[5.68434189e-14 5.68434189e-14 5.68434189e-14 5.68434189e-14
  5.68434189e-14]
 [5.68434189e-14 5.68434189e-14 5.68434189e-14 5.68434189e-14
  5.68434189e-14]
 [5.68434189e-14 5.68434189e-14 5.68434189e-14 5.68434189e-14
  5.68434189e-14]
 [5.68434189e-14 5.68434189e-14 5.68434189e-14 5.68434189e-14
  5.68434189e-14]
 [5.68434189e-14 5.68434189e-14 5.68434189e-14 5.68434189e-14
  5.68434189e-14]]


Is your bound still valid using Monte Carlo estimation for the expectation ? Why ?

> **Answer:** The bound is derived assuming that the expectation is computed exactly. However, when using MC estimates, we actually introduce additional estimation error (variance) because we are approximating the true expectation with a finite number of samples. However, as the number of MC samples increases, the estimate converges to the true expectation, and the bound will hold asymptotically. But here, we only do 10 MC runs which we can assume to be clearly insufficient, so we have to account for the extra error due to sampling variance.
> 
> So **no**, the bound is not valid anymore.

## Bonus: 
How can you adjust your implementation if you know the dynamics of the environment to compute the exact $V^\mu_N$ function ?
Implement your solution.

In [9]:
def truncated_value_function(env, policy, gamma=0.99, N=100):
    """
    Estimate the state value function in the stochastic grid world environment over a specified number of iterations.

    This function calculates the expected value of each state in the environment when following a given policy,
    using the dynamics of the environment to account for stochasticity in the environment.

    Parameters:
    env (Env): The environment in which the policy is evaluated.
    policy (function): A function that takes a state as input and returns an action according to the policy being evaluated.
    gamma (float, optional): The discount factor, which determines the present value of future rewards. Default is 0.99.
    N (int, optional): The number of iterations to perform for estimating the value function. Default is 100.

    Returns:
    tuple: A tuple containing:
        - state_value_function_table (np.ndarray): An array representing the estimated value of each state.
        - std_state_value_function_table (np.ndarray): An array representing the standard deviation of the estimated value for each state.
    """
    state_value_function_table = np.zeros((env.n, env.m))
    std_state_value_function_table = np.zeros((env.n, env.m))

    for _ in range(N):
        new_state_value_function_table = np.zeros((env.n, env.m))
        for i in range(env.n):
            for j in range(env.m):
                s = (i, j)
                a = policy(s)
                s_next, r, _, _, _ = env.step(a)
                x, y = s_next
                value = r + gamma * state_value_function_table[x, y]
                new_state_value_function_table[i, j] = value
        state_value_function_table = new_state_value_function_table

    return state_value_function_table, std_state_value_function_table

# Example usage
truncated_value_function_table, std_truncated_value_function_table = truncated_value_function(env, rule_based_policy)
print("Truncated V with N=100:")
print(truncated_value_function_table)
print(std_truncated_value_function_table)

Truncated V with N=100:
[[-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]
 [-316.98382936 -316.98382936 -316.98382936 -316.98382936 -316.98382936]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
